# RNA-Puzzles Assessment example

> This notebook provides a tutorial on RNA 3D structure comparison in RNA-Puzzles

In [1]:
import sys,os
# import warnings
# warnings.filterwarnings("ignore")

import RNA_normalizer

from operator import attrgetter

RESIDUES_LIST = "data/residues.list"
ATOMS_LIST = "data/atoms.list"

def CleanFormat(f):
	os.system( "mac2unix -q %s" %f )
	os.system( "dos2unix -q %s" %f )

> CleanFormat is a function used to format different platform formats to unix. Users need to install dos2unix

# Normalize structures
> Different prediction methods may give different PDB formats. The standard PDB format considered is the 
[1992](https://www.rcsb.org/pdb/file_formats/pdb/pdbguide2.2/PDB_format_1992.pdf) format. To normalize the structure format, we need to:   
* Correct the residue names and the atom names: The name mapping dictionaries are: defined as "RESIDUES_LIST" and "ATOMS_LIST" in the data folder. 
* select the part of structure we are interested. The part of structure is assigned in "index_file". The selection grammar is: [Chain id]:[Beg of Residue number]:[length of fragment]
    + [Chain id] is single letter character
    + [Beg of Residue number] is the number of the beginning residue in the PDB file. This number is exact the "res. seq no." field of the PDB file without any renumbering. 
    + [length of fragment] is the length of the fragment. numbering of the residues in the fragment is not considered.
    + for example, A:1:31,A:33:29 includes two fragments of 31 residue and 29 residues. The first fragment starts from residue 1 and until residue 31. leaving out residue 32, the second fragment starts from residue 33 and ends till 29 residues after. The number of these 29 residues are not considered except the first one. 
* extract the interesting part. To do this, we need to set the output file for "extract_file" parameter. 

In [2]:
def normalize_structure(struct, out_file = None, index_file=None, extract_file = None):
	pdb_normalizer = RNA_normalizer.PDBNormalizer( RESIDUES_LIST, ATOMS_LIST )
	ok = pdb_normalizer.parse( struct, out_file )
	if not ok:
		sys.stderr.write("ERROR: structure not normalized!\n")
	else:
		sys.stderr.write("INFO: Normalization succeded!\n")
	if not extract_file is None:
		coords=open(index_file).read()
		RNA_normalizer.extract_PDB(SOLUTION_NORMAL,coords, extract_file)
		sys.stderr.write("INFO:	structure extracted\n")

# Calculate RMSD and P-value
> This function calculate the all atom RMSD and the P-value of prediction. 
* Before calculation, we need to first make sure that the two structure have the same sequence. 
If we need to use the whole structure instead of the selected part assigned by index_file, we can give an empty index_file.
* The RMSD calculated is all atom RMSD.
* the P value set is according to ***Hajdin et al., RNA (7) 16, 2010***. By default, it is set to "-".

In [3]:
# PVALUE set according to Hajdin et al., RNA (7) 16, 2010, either "+" or "-"
def calc_RMSD(native_file, native_index, prediction_file, prediction_index, PVALUE = "-"):
	res_struct = RNA_normalizer.PDBStruct()
	res_struct.load( native_file, native_index )
	res_raw_seq = res_struct.raw_sequence()
	
	sol_struct = RNA_normalizer.PDBStruct()
	sol_struct.load( prediction_file, prediction_index )
	sol_raw_seq = sol_struct.raw_sequence()
	
	if( sol_raw_seq != res_raw_seq ):
		sys.stderr.write("ERROR Result sequence != Solution sequence!\n")
		sys.stderr.write("DATA Solution sequence --> '%s'\n" %sol_raw_seq )
		sys.stderr.write("DATA Result sequence   --> '%s'\n" %res_raw_seq )
		return(-1)
	# computes the RMSD
	comparer = RNA_normalizer.PDBComparer()
	rmsd = comparer.rmsd( sol_struct, res_struct )
	sys.stderr.write("INFO Partial RMSD --> %f\n" %rmsd )
	pvalue = comparer.pvalue( rmsd, len(sol_raw_seq), PVALUE )
	sys.stderr.write("INFO Partial P-Value --> %e\n" %pvalue )
	return(rmsd, pvalue)

# Interaction Network Fidelity
> Interaction Network Fidelity is defined by ***RNA. 2009 Oct; 15(10): 1875–1885.***.
* It checks the percentage of interactions that are found in the predicted structure. 
* The interaction types are assigned according to [MC-annotate](http://www-lbit.iro.umontreal.ca/mcannotate-simple/). 
* The interaction types can be sorted to: All interactions (INF_ALL), Watson-Crick interactin (INF_WC), non-Watson-Crick interaction 
   (INF_NWC) and Stacking (INF_STACK). 
* Deformation Index is defined as RMSD/INF_ALL (***RNA. 2009 Oct; 15(10): 1875–1885.***). 
* This function returns a tuple of the metrices [RMSD, Deformation Index, INF_ALL, INF_WC, INF_NWC,INF_STACK]

In [4]:
def InteractionNetworkFidelity(native_file, native_index, prediction_file, prediction_index):
	res_struct = RNA_normalizer.PDBStruct()
	res_struct.load( native_file, native_index )
	res_raw_seq = res_struct.raw_sequence()
	
	sol_struct = RNA_normalizer.PDBStruct()
	sol_struct.load( prediction_file, prediction_index )
	sol_raw_seq = sol_struct.raw_sequence()
	
	if( sol_raw_seq != res_raw_seq ):
		sys.stderr.write("ERROR Result sequence != Solution sequence!\n")
		sys.stderr.write("DATA Solution sequence --> '%s'\n" %sol_raw_seq )
		sys.stderr.write("DATA Result sequence   --> '%s'\n" %res_raw_seq )
		return(-1)
	# computes the RMSD
	comparer = RNA_normalizer.PDBComparer()
	rmsd = comparer.rmsd( sol_struct, res_struct )
	INF_ALL = comparer.INF( sol_struct, res_struct, type="ALL" )
	DI_ALL = rmsd / INF_ALL
	INF_WC = comparer.INF( sol_struct, res_struct, type="PAIR_2D" )
	INF_NWC = comparer.INF( sol_struct, res_struct, type="PAIR_3D" )
	INF_STACK = comparer.INF( sol_struct, res_struct, type="STACK" )
	return (rmsd,DI_ALL, INF_ALL, INF_WC, INF_NWC,INF_STACK)

# The Local Distance Difference Test (lDDT)
> The Local Distance Difference Test (lDDT) is defined by ***Bioinformatics. 2013 Nov 1;29(21):2722-8.***.
* It checks the percentage of interactions that are found in the predicted structure. 
* [OpenStructure](https://openstructure.org/download) for lDDT calculations.
* This function returns the lDDT score

In [5]:
import os
#import extract
import sys
sys.path.append('./RNA_normalizer')
sys.path.append('./lddt/bin')
import extract

from structure_processed import preprocess
from metrics_lddt import LDDT

def extract_and_preprocess(parent_dir, pdb_list):
    # extract and process PDB
    for pdb_name in pdb_list:
        coords = ""
        index_file_path = os.path.join(parent_dir, f"{pdb_name.split('.pdb')[0]}.index")
        with open(index_file_path, "r") as index_file:
            for row in index_file:
                row = row.strip()
                if not row.startswith("#") and row:
                    coords += row + ","  
        coords = coords.rstrip(",") 
        
        pdb_path = os.path.join(parent_dir, pdb_name)
        new_pdb_path = os.path.join(parent_dir, f"{pdb_name.split('.pdb')[0]}_extract.pdb")
        extract.extract_PDB(pdb_path, coords, new_pdb_path)
        preprocess(new_pdb_path)  # run on the new pdb structure

def calculate_lddt(reference, model,chain_mapping,force=False):
    # calculate the lDDT score
    lddt = LDDT()
    lddt_output=lddt.calculate(reference, model,chain_mapping,force=force)
    return lddt_output
    
    

# call the functions

In [6]:
# %%capture

# Normalize PDB format, correct residue names and atom names. 
normalize_structure('example/PZ14Bound_solution_4.pdb','example/PZ14Bound_solution_normalized.pdb')

# calculate RMSD for RNA structures
# require biopython
rmsd,pval = calc_RMSD("example/PZ14Bound_solution_4.pdb",
          "example/PZ14Bound_solution_4.index",
          "example/14_ChenPostExp_2.pdb",
          "example/14_ChenPostExp_2.index")

# calculate InteractionNetworkFidelity and Deformation Index for RNA structures
# need to have MA-annotate in the directory or set in mcannotate.py
rmsd2,DI_ALL, INF_ALL, INF_WC, INF_NWC,INF_STACK = InteractionNetworkFidelity("example/PZ14Bound_solution_4.pdb",
          "example/PZ14Bound_solution_4.index",
          "example/14_ChenPostExp_2.pdb",
          "example/14_ChenPostExp_2.index")

INFO: Normalization succeded!
WARNING    >> Atom P from residue (' ', 1, ' ') not found in target atom list
WARNING    >> Atom OP1 from residue (' ', 1, ' ') not found in target atom list
WARNING    >> Atom OP2 from residue (' ', 1, ' ') not found in target atom list
INFO Partial RMSD --> 7.640878
INFO Partial P-Value --> 4.551914e-15
WARNING    >> Atom P from residue (' ', 1, ' ') not found in target atom list
WARNING    >> Atom OP1 from residue (' ', 1, ' ') not found in target atom list
WARNING    >> Atom OP2 from residue (' ', 1, ' ') not found in target atom list


In [7]:
parent_dir = '/home/xxx/RNA_assessment/example/'
pdb_list = ['14_ChenPostExp_2.pdb', '14_solution_0.pdb']

# first step: extract and process
extract_and_preprocess(parent_dir, pdb_list)

# construct the name of the new file(_processed.pdb) 
reference = os.path.join(parent_dir, 'processed', '14_solution_0_extract_processed.pdb')
model = os.path.join(parent_dir, 'processed', '14_ChenPostExp_2_extract_processed.pdb')
lDDT=calculate_lddt(reference, model,chain_mapping='{"A": "A"}')

Processing /home/bufan/RNA_assessment/example/14_ChenPostExp_2_extract.pdb
Did not create new folder.
/home/bufan/RNA_assessment/example/processed already exists.
Processing /home/bufan/RNA_assessment/example/14_solution_0_extract.pdb
Did not create new folder.
/home/bufan/RNA_assessment/example/processed already exists.
command: /home/bufan/RNA_assessment/lddt/ema --mount /home/bufan/RNA_assessment/example/processed /home/bufan/RNA_assessment/lddt/bin/complex_lddt_no_stereocheck.py /home/bufan/RNA_assessment/example/processed/14_ChenPostExp_2_extract_processed.pdb /home/bufan/RNA_assessment/example/processed/14_solution_0_extract_processed.pdb '{"A": "A"}'


# Show the results:

In [8]:
'RMSD: %.3f; P-value: %.3e; Deformation Index: %.3f; \
INF_all: %.3f; INF_wc: %.3f; INF_nwc: %.3f; INF_stack: %.3f,lDDT: %.3f'%(rmsd,pval,DI_ALL, INF_ALL, INF_WC, INF_NWC,INF_STACK,lDDT)

'RMSD: 7.641; P-value: 4.552e-15; Deformation Index: 10.567; INF_all: 0.723; INF_wc: 0.938; INF_nwc: 0.250; INF_stack: 0.701,lDDT: 0.548'